<a href="https://colab.research.google.com/github/Marcusreu1/Natural-language-review-positive-and-negative-/blob/main/Natural_language_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import libraries

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np



#descargar data

In [2]:
vocab_size=88584

maxlen=250
batch_size=64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words= vocab_size)

17464789/17464789 [==============================] - 1s 0us/step


#preprocessing

In [3]:
train_data=sequence.pad_sequences(train_data,maxlen)
test_data=sequence.pad_sequences(test_data,maxlen)

#creating the model

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(88584, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#see model

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#train model

In [6]:
model.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop",
    metrics=["acc"]
    )

history= model.fit(train_data,
                   train_labels,
                   epochs=10,
                   validation_split=0.2
                   )

Epoch 1/10
625/625 [==============================] - 63s 97ms/step - loss: 0.4469 - acc: 0.7847 - val_loss: 0.3698 - val_acc: 0.8386
Epoch 2/10
625/625 [==============================] - 57s 92ms/step - loss: 0.2611 - acc: 0.8992 - val_loss: 0.2920 - val_acc: 0.8750
Epoch 3/10
625/625 [==============================] - 64s 103ms/step - loss: 0.2035 - acc: 0.9247 - val_loss: 0.3003 - val_acc: 0.8838
Epoch 4/10
625/625 [==============================] - 70s 112ms/step - loss: 0.1615 - acc: 0.9411 - val_loss: 0.2932 - val_acc: 0.8812
Epoch 5/10
625/625 [==============================] - 57s 91ms/step - loss: 0.1357 - acc: 0.9529 - val_loss: 0.3158 - val_acc: 0.8796
Epoch 6/10
625/625 [==============================] - 62s 99ms/step - loss: 0.1147 - acc: 0.9603 - val_loss: 0.3190 - val_acc: 0.8844
Epoch 7/10
625/625 [==============================] - 58s 93ms/step - loss: 0.0974 - acc: 0.9683 - val_loss: 0.3915 - val_acc: 0.8778
Epoch 8/10
625/625 [==============================] - 59s 95

#evaluar resultados

In [7]:
results= model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 18s 24ms/step - loss: 0.5274 - acc: 0.8563
[0.5274378657341003, 0.8563200235366821]


#codificar texto a numeros

In [8]:
import keras.preprocessing

word_index=imdb.get_word_index()


def encode_text(text):
  tokens= keras.preprocessing.text.text_to_word_sequence(text)
  tokens=[word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], maxlen)[0]


text="that movie was just amazing, so amazing"
encoded= encode_text(text)
print(encoded)

1641221/1641221 [==============================] - 1s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

#descodificar numeros a texto

In [10]:
reverse_word_index={value:key for (key, value) in word_index.items()}


def decode_integers(integers):
  Pad= 0
  text=""
  for num in integers:
    if num !=Pad:
      text += reverse_word_index[num]+ " "
  return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


#hacer prediccion

In [21]:
def predict(text):
  encoded_text= encode_text(text)
  pred= np.zeros((1,250))
  pred[0]= encoded_text
  result= model.predict(pred)
  print(result[0])


positive_review="that movie was amazing men"
predict(positive_review)

negative_review="that movie is so bad"
predict(negative_review)

1/1 [==============================] - 0s 25ms/step
[0.6314086]
1/1 [==============================] - 0s 27ms/step
[0.46635795]
